In [ ]:
import csv
import email
import io
import os
import re
import smtplib
import ssl
import subprocess

from collections import namedtuple
from email import encoders
from email.mime.application import MIMEApplication
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from functools import partial
from pprint import pprint
from string import Template

from more_itertools import pairwise
from IPython.display import Markdown
from io import StringIO

In [ ]:
APP_PASSWORD = "cjlzkelypzyvjinf"

COLLEGES_LIST = os.path.expanduser("~/Repos/academic-job-search-2021/academic-job-search-2021.csv")
COLLEGES_LIST_HEADINGS = 'Updated, Do_Not_Contact, School, Department, Chair, Title, Email, Salutation, Source, Notes'.split(", ")
MESSAGES_DIRECTORY_PATH = os.path.expanduser("~/Repos/academic-job-search-2021/messages")
CV_PATH = os.path.expanduser("~/OneDrive/academic-jobs/academic-jobs-cv/Wattenbarger--CV--20210225.docx")
DOCX_MIME_SUBTYPE = "vnd.openxmlformats-officedocument.wordprocessingml.document"
JOB_SEARCH_SUBJECT_LINE = "Seeking music appreciation and music history opportunities for 2021/22"
NON_WORD_CHARS_RE = re.compile(r"[\.,']")

In [ ]:
def read_lines(filename):
    with open(filename) as fp:
        return [line[:-1] for line in fp.readlines()]  
    
def is_white_space(line):
    return len(line.strip()) == 0
    
def convert_to_markdown(filename):
    lines = list(read_lines(filename))
    with StringIO() as fp:
        for line1, line2 in pairwise(lines):
            if not is_white_space(line1) and not is_white_space(line2):
                    fp.write(f'{line1}<br>\n')
            else:
                    fp.write(f'{line1}\n')
        fp.write(f'{lines[-1]}')
        return fp.getvalue()

def convert_to_html(filename):
    markdown_text = convert_to_markdown(filename)
    cp = subprocess.run(['pandoc', '-f', 'gfm', '-s', '--metadata', 'title=" "', '--template=jobsearch'], 
                        input=markdown_text, capture_output=True, text=True)
    if cp.returncode == 0:
        return cp.stdout
    else:
        print(cp.stderr)
    
def change_ext(filename, new_ext):
    basename, _ = os.path.splitext(filename)
    if new_ext[0] != ".":
        new_ext = f'.{new_ext}'
    return f'{basename}{new_ext}'
        
change_ext_to_html = partial(change_ext, new_ext=".html")

def write_message_file(filename, message):
    with open(filename, mode='w') as fp:
        fp.write(message)
        
def build_filename(message_key):
    return NON_WORD_CHARS_RE.sub('', message_key.lower()).replace(' ','-') + ".txt"

def read_csv(filename, headings):
    with open(filename, newline='') as fp:
        reader = csv.DictReader(fp, fieldnames=headings)
        return [row for row in reader][1:]

def read_message_file(filename):
    with open(filename) as fp:
        return fp.read()

def get_plain_text_message(filename):
    return MIMEText(read_message_file(filename), "plain")

def change_extension(filename, new_ext):
    basename, _ = os.path.splitext(filename)
    if new_ext[0] != ".":
        new_ext = f".{new_ext}"
    return f'{basename}{new_ext}'

def read_cv(filename):
    with open(filename, "rb") as fp:
        return fp.read()
    
def get_attachment_part(filename : str, mime_subtype : str=DOCX_MIME_SUBTYPE) -> MIMEApplication:
    part = MIMEApplication(read_cv(filename), _subtype=mime_subtype)
    part.add_header("Content-Disposition", "attachment", filename=filename)
    return part

def set_message_header_values(message : MIMEMultipart, message_header_fields : dict) -> None:
    for key, value in message_header_fields.items():
        message[key] = value

def build_message(plain_text_filename : str,
                  attachment_path : str,
                  message_header_fields : dict) -> str:
    
    message_alt = MIMEMultipart("alternative")

    html_text = convert_to_html(plain_text_filename)
    message_alt.attach(MIMEText(html_text, 'html'))
    message_alt.attach(get_plain_text_message(plain_text_filename))

    message_mixed = MIMEMultipart('mixed')
    message_mixed.attach(message_alt)
    message_mixed.attach(get_attachment_part(CV_PATH))
    set_message_header_values(message_mixed, message_header_fields)
    
    return message_mixed.as_string()

def send_message(message_text, recipients):
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login('rwattenb@gmail.com', APP_PASSWORD)
        server.sendmail('richard@wattenbarger.net', recipients, message_text)

def process_mailing(recipients, plain_text_message_path, html_message_path, attachment_path, message_header_fields):
    message = build_message(plain_text_message_filename, html_message_filename, attachment_path, message_header_fields)
    send_message(message, recipients=recipients)   
    
def send_messages():
    rows = [row for row in read_csv(COLLEGES_LIST, COLLEGES_LIST_HEADINGS) 
            if row['Do_Not_Contact'] != 'Y' and row['Updated'] == 'Y' and row['Chair']]
    for row in rows:
        recipients = row["Email"]
        print(f'sending message for {row["School"]}')
#         recipients =  "Richard Wattenbarger <rwattenb@hotmail.com>" 
        plain_text_message_filename = os.path.join(MESSAGES_DIRECTORY_PATH, build_filename(row['School']))
#         html_message_filename = change_ext_to_html(plain_text_message_filename)
        message_header_fields = {"From":"Richard Wattenbarger <richard@wattenbarger.net"
                                ,"To":recipients
                                ,"Subject": JOB_SEARCH_SUBJECT_LINE
                                ,"Reply-To": "Richard Wattenbarger <richard@wattenbarger.net>"}
        message = build_message(plain_text_message_filename, CV_PATH, message_header_fields)
        send_message(message, recipients=recipients)
    
send_messages()

In [ ]:
html_text = convert_to_html(os.path.join(MESSAGES_DIRECTORY_PATH, 'arcadia-university.txt'))
print(html_text)

In [ ]:
def display_input_data(rows):
    with StringIO() as fp:
        keys = rows[0].keys()
        for key in keys:
            fp.write(f"| {key} ")
        fp.write("|\n")
        for i in range(len(keys)):
            fp.write("| - ")
        fp.write("|\n")

        for row in rows:
            for key in keys:
                fp.write(f"| {row[key]} ")
            fp.write("|\n")
        
        raw_markdown = fp.getvalue()
    
    display(Markdown(data=raw_markdown))
    
display_input_data(read_csv(COLLEGES_LIST, COLLEGES_LIST_HEADINGS))

In [ ]:
print(os.path.dirname(os.path.join(MESSAGES_DIRECTORY_PATH, 'arcadia-university.html')))
dir(os.path)

In [ ]:
message_template_text = """From: ${from}
To: ${to}
Subject ${subject}

${body}

Regards,

RW"""

template = Template(message_template_text)

message_text = template.substitute({"from":"rwattenb@gmail.com", 
                                    "to":"rwattenb@gmail.com", 
                                    "subject":"test", 
                                    "body":"this is only a test."})

print(message_text)

In [ ]:


try:
#     server = smtplib.SMTP('smtp.gmail.com', 587)
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
#     server.starttls()
    server.login('rwattenb@gmail.com', APP_PASSWORD)
    server.sendmail('rwattenb@gmail.com', ['rwattenb@gmail.com'], message_text)
    server.quit()
except ex:
    print(ex)
    

In [ ]:
def send_to_hillmont() -> None:
    PLAIN_TEXT = os.path.expanduser("~/temp/medications.txt")
    HTML_TEXT = os.path.expanduser("~/temp/medications.html")
    
    message_header_fields = MessageHeaderFields(
        sender = "Richard Wattenbarger <richard@wattenbarger.net>", 
        recipients = "jtreichel@hillmontgi.com",
        subject = "Medications for Richard Wattenbarger",
        reply_to = "richard@wattenbarger.net")
    
    message_text = build_message(message_header_fields, PLAIN_TEXT, HTML_TEXT)
    send_messsage(message_text)
    
send_to_hillmont()